# Deteccion areas quemedas historico

En este codigo se realiza el procedimiento para la deteccion de areas quemadas en un periodo de tiempo dado. este se realiza por medio de bloques cada x dias para evitar problemas de memoria o tiempo. El script funciona con autenticacion de google earth engine asi que se tiene que crear un usuario y projecto con acceso a la API de google earth engine en google cloud.

In [2]:
import ee
import datetime
from dateutil.relativedelta import relativedelta
import time
import os

ee.Authenticate(force=True)
ee.Initialize(project='incendios-461918')


class QuemadaDetector:
    def __init__(self, region_asset_path, max_dias_busqueda=1, num_imagenes_previas=5, 
                 nubes_max_objetivo=100, nubes_max_previas=50, dias_por_bloque=10):
        """
        Inicializar el detector de áreas quemadas
        
        Args:
            region_asset_path (str): Ruta del asset de la región en GEE
            max_dias_busqueda (int): Máximo de días para buscar imagen si no hay en fecha exacta
            num_imagenes_previas (int): Número de imágenes previas para comparar
            nubes_max_objetivo (float): % máximo de nubes para imágenes objetivo
            nubes_max_previas (float): % máximo de nubes para imágenes previas
            dias_por_bloque (int): Número de días por bloque de procesamiento
        """
        self.region = ee.FeatureCollection(region_asset_path)
        self.max_dias_busqueda = max_dias_busqueda
        self.num_imagenes_previas = num_imagenes_previas
        self.nubes_max_objetivo = nubes_max_objetivo
        self.nubes_max_previas = nubes_max_previas
        self.dias_por_bloque = dias_por_bloque
        
        # Parámetros de umbrales
        self.umbral_diff_ndvi = -0.3
        self.umbral_ndvi_prev = 0.5
        self.umbral_bais2 = 6
        self.umbral_mirbi = 0
        
        # Colección base para imágenes objetivo
        self.coleccion_objetivo = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterBounds(self.region) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', self.nubes_max_objetivo)) \
            .map(self.mask_s2_clouds) \
            .map(lambda img: img.clip(self.region))
        
        # Colección base para imágenes previas
        self.coleccion_previas = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterBounds(self.region) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', self.nubes_max_previas)) \
            .map(self.mask_s2_clouds) \
            .map(lambda img: img.clip(self.region))
        
        print(f"Detector inicializado. Región cargada con {self.region.size().getInfo()} features")
        print(f"Filtro nubes objetivo: < {self.nubes_max_objetivo}%")
        print(f"Filtro nubes previas: < {self.nubes_max_previas}%")
        print(f"Días por bloque: {self.dias_por_bloque}")
    
    def mask_s2_clouds(self, image):
        """Función mejorada para enmascarar nubes (QA60 + SCL)"""
        # 1. Máscara QA60 (nubes y cirros)
        qa = image.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        mask_qa = qa.bitwiseAnd(cloud_bit_mask).eq(0) \
                   .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        
        # 2. Máscara SCL: nubes medias, altas, sombras y cirros delgados
        scl = image.select('SCL')
        mask_scl = scl.neq(3) \
                     .And(scl.neq(8)) \
                     .And(scl.neq(9)) \
                     .And(scl.neq(10))
        
        # Combinar máscaras
        mask_final = mask_qa.And(mask_scl)
        
        return image.updateMask(mask_final)
    
    def calcular_ndvi(self, image):
        """Calcular NDVI usando las bandas B8 (NIR) y B4 (rojo)"""
        return image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    def calcular_bais2(self, image):
        """Calcular índice BAIS2 para detección de áreas quemadas"""
        b4 = image.select('B4')
        b6 = image.select('B6')
        b7 = image.select('B7')
        b8a = image.select('B8A')
        b12 = image.select('B12')
        
        bais2 = image.expression(
            '(1 - sqrt((RE2 * RE3 * N2) / R)) * (sqrt((S2 - N2) / (S2 + N2)) + 1)', {
                'RE2': b6,
                'RE3': b7,
                'N2': b8a,
                'R': b4,
                'S2': b12
            }
        ).rename('BAIS2')
        
        return bais2
    
    def calcular_mirbi(self, image):
        """Calcular MIRBI para detectar áreas quemadas"""
        return image.expression(
            '10 * (B12 - (B11 + B8A) / 2)', {
                'B12': image.select('B12'),
                'B11': image.select('B11'),
                'B8A': image.select('B8A')
            }
        ).rename('MIRBI')
    
    def buscar_mosaico_en_fecha(self, fecha_objetivo, coleccion):
        """
        Buscar y crear mosaico para una fecha específica, avanzando día a día si es necesario
        
        Args:
            fecha_objetivo (ee.Date): Fecha objetivo
            coleccion (ee.ImageCollection): Colección donde buscar
            
        Returns:
            tuple: (mosaico, fecha_real, num_imagenes, encontrada)
        """
        current_date = fecha_objetivo
        
        for i in range(self.max_dias_busqueda):
            start_date = current_date
            end_date = current_date.advance(1, 'day')
            
            img_collection = coleccion.filterDate(start_date, end_date)
            num_imagenes = img_collection.size().getInfo()
            
            # Verificar si hay imágenes
            if num_imagenes > 0:
                fecha_real = current_date.format('YYYY-MM-dd')
                
                if num_imagenes == 1:
                    # Si solo hay una imagen, usarla directamente
                    mosaico = ee.Image(img_collection.first())
                else:
                    # Si hay múltiples imágenes, crear mosaico
                    mosaico = img_collection.mosaic()
                
                # Agregar metadatos al mosaico
                mosaico = mosaico.set({
                    'fecha_mosaico': fecha_real,
                    'num_imagenes_mosaico': num_imagenes,
                    'system:time_start': current_date.millis()
                })
                
                return mosaico, fecha_real, num_imagenes, True
            
            current_date = current_date.advance(1, 'day')
        
        return None, None, 0, False
    
    def crear_mosaicos_por_fecha(self, coleccion, num_fechas):
        """
        Crear mosaicos para las N fechas más recientes, agrupando imágenes por fecha
        
        Args:
            coleccion (ee.ImageCollection): Colección de imágenes
            num_fechas (int): Número de fechas únicas a obtener
            
        Returns:
            list: Lista de mosaicos únicos por fecha
        """
        # Obtener lista de fechas únicas
        def extract_date(image):
            return ee.Feature(None, {
                'date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
                'timestamp': image.get('system:time_start')
            })
        
        # Extraer fechas de todas las imágenes
        fechas_collection = coleccion.map(extract_date)
        fechas_distintas = fechas_collection.distinct(['date']).sort('timestamp', False)
        
        # Tomar solo las N fechas más recientes
        fechas_lista = fechas_distintas.limit(num_fechas).aggregate_array('date')
        fechas_info = fechas_lista.getInfo()
        
        print(f"      Fechas únicas encontradas: {len(fechas_info)} de {num_fechas} solicitadas")
        
        mosaicos = []
        
        for fecha_str in fechas_info:
            # Filtrar imágenes de esta fecha específica
            fecha_ee = ee.Date(fecha_str)
            inicio_dia = fecha_ee
            fin_dia = fecha_ee.advance(1, 'day')
            
            imagenes_fecha = coleccion.filterDate(inicio_dia, fin_dia)
            num_imagenes = imagenes_fecha.size().getInfo()
            
            if num_imagenes > 0:
                if num_imagenes == 1:
                    # Si solo hay una imagen, usarla directamente
                    mosaico = ee.Image(imagenes_fecha.first())
                    print(f"        - {fecha_str}: 1 imagen")
                else:
                    # Si hay múltiples imágenes, crear mosaico
                    mosaico = imagenes_fecha.mosaic()
                    print(f"        - {fecha_str}: {num_imagenes} imágenes → mosaico")
                
                # Agregar metadatos al mosaico
                mosaico = mosaico.set({
                    'fecha_mosaico': fecha_str,
                    'num_imagenes_mosaico': num_imagenes,
                    'system:time_start': fecha_ee.millis()
                })
                
                mosaicos.append(mosaico)
        
        return mosaicos
    
    def procesar_bloque(self, fecha_inicio, fecha_fin):
        """
        Procesar un bloque específico de días
        
        Args:
            fecha_inicio (datetime.date): Fecha de inicio del bloque
            fecha_fin (datetime.date): Fecha de fin del bloque
            
        Returns:
            list: Lista de vectores del bloque
        """
        print(f"  📅 Procesando bloque: {fecha_inicio} a {fecha_fin}")
        
        vectores_bloque = []
        imagenes_procesadas = set()
        
        # Procesar cada día del bloque
        current_date = fecha_inicio
        while current_date <= fecha_fin:
            fecha_str = current_date.strftime('%Y-%m-%d')
            fecha_ee = ee.Date(fecha_str)
            
            # Buscar y crear mosaico para la fecha objetivo
            mosaico_objetivo, fecha_real_ee, num_imgs_objetivo, encontrada = self.buscar_mosaico_en_fecha(fecha_ee, self.coleccion_objetivo)
            
            if not encontrada:
                current_date += datetime.timedelta(days=1)
                continue
            
            # Obtener fecha real como string
            fecha_real = fecha_real_ee.getInfo()
            
            # Evitar procesar la misma fecha varias veces
            if fecha_real in imagenes_procesadas:
                print(f"    Mosaico {fecha_real} ya procesado, omitiendo {fecha_str}")
                current_date += datetime.timedelta(days=1)
                continue
            
            imagenes_procesadas.add(fecha_real)
            
            # Mostrar información del mosaico objetivo
            if num_imgs_objetivo == 1:
                print(f"    Procesando mosaico objetivo del {fecha_real} (1 imagen)")
            else:
                print(f"    Procesando mosaico objetivo del {fecha_real} ({num_imgs_objetivo} imágenes)")
            
            # Para el % de nubes, calculamos un promedio ponderado aproximado
            # (En un mosaico esto es más complejo, pero usamos el valor máximo permitido como referencia)
            nubes_objetivo = self.nubes_max_objetivo  # Valor representativo para mosaicos
            
            # Calcular índices para mosaico objetivo
            ndvi_objetivo = self.calcular_ndvi(mosaico_objetivo)
            bais_objetivo = self.calcular_bais2(mosaico_objetivo)
            mirbi_objetivo = self.calcular_mirbi(mosaico_objetivo)
            
            # Obtener imágenes previas y crear mosaicos por fecha
            target_millis = ee.Number(mosaico_objetivo.get('system:time_start'))
            imagenes_previas_collection = self.coleccion_previas \
                .filter(ee.Filter.lt('system:time_start', target_millis)) \
                .sort('system:time_start', False)
            
            # Verificar si hay imágenes previas
            if imagenes_previas_collection.size().getInfo() == 0:
                print(f"    No hay imágenes previas para {fecha_real}")
                current_date += datetime.timedelta(days=1)
                continue
            
            # Crear mosaicos por fecha
            print(f"    Creando mosaicos para {self.num_imagenes_previas} fechas previas...")
            mosaicos_previos = self.crear_mosaicos_por_fecha(imagenes_previas_collection, self.num_imagenes_previas)
            
            if len(mosaicos_previos) == 0:
                print(f"    No se pudieron crear mosaicos previos para {fecha_real}")
                current_date += datetime.timedelta(days=1)
                continue
            
            print(f"    Creados {len(mosaicos_previos)} mosaicos previos")
            
            # Procesar cada mosaico previo
            for mosaico_prev in mosaicos_previos:
                fecha_prev = mosaico_prev.get('fecha_mosaico').getInfo()
                num_imgs_mosaico = mosaico_prev.get('num_imagenes_mosaico').getInfo()
                
                # Para el % de nubes, usamos un valor representativo
                nubes_prev = self.nubes_max_previas  # Valor por defecto para mosaicos
                
                # Calcular NDVI del mosaico previo
                ndvi_prev = self.calcular_ndvi(mosaico_prev)
                
                # Calcular diferencia de NDVI
                diff_ndvi = ndvi_objetivo.subtract(ndvi_prev).rename('Diferencia_NDVI')
                
                # Crear máscara inicial: zonas donde diff NDVI < umbral
                zonas = diff_ndvi.lt(self.umbral_diff_ndvi).selfMask()
                zonas_clip = zonas.clip(self.region)
                
                # Convertir a vectores para obtener geometría de áreas de interés
                try:
                    vectores_temp = zonas_clip.reduceToVectors(
                        geometry=self.region,
                        scale=12.5,
                        geometryType='polygon',
                        eightConnected=True,
                        labelProperty='zone',
                        maxPixels=1e8
                    )
                    
                    if vectores_temp.size().getInfo() == 0:
                        continue
                    
                    # Unir polígonos
                    union_poligono = vectores_temp.geometry()
                    if union_poligono.area(maxError=1).getInfo() == 0:
                        continue
                    
                    poligono_fc = ee.FeatureCollection([ee.Feature(union_poligono)])
                    
                    # Aplicar máscara combinada
                    combined_mask = bais_objetivo.lt(self.umbral_bais2) \
                                   .Or(mirbi_objetivo.gt(self.umbral_mirbi)) \
                                   .And(ndvi_prev.gt(self.umbral_ndvi_prev)) \
                                   .selfMask() \
                                   .rename('Combined_mask')
                    
                    # Recortar a área de interés
                    image_clip_union = combined_mask.clip(poligono_fc)
                    
                    # Vectorizar resultado final
                    vectores_final = image_clip_union.reduceToVectors(
                        geometry=self.region,
                        scale=12.5,
                        geometryType='polygon',
                        eightConnected=True,
                        maxPixels=1e8
                    )
                    
                    # Agregar metadatos
                    vectores_final = vectores_final.map(lambda feature: 
                        feature.set({
                            'fecha_objetivo': fecha_real,
                            'fecha_previa': fecha_prev,
                            'nubes_objetivo': nubes_objetivo,
                            'nubes_previa': nubes_prev,
                            'imagenes_objetivo': num_imgs_objetivo,
                            'imagenes_mosaico': num_imgs_mosaico,
                            'bloque_inicio': fecha_inicio.strftime('%Y-%m-%d'),
                            'bloque_fin': fecha_fin.strftime('%Y-%m-%d')
                        })
                    )
                    
                    vectores_bloque.append(vectores_final)
                    num_poligonos = vectores_final.size().getInfo()
                    
                    # Mostrar información detallada
                    objetivo_info = f"mosaico {num_imgs_objetivo} imgs" if num_imgs_objetivo > 1 else "1 imagen"
                    previa_info = f"mosaico {num_imgs_mosaico} imgs" if num_imgs_mosaico > 1 else "1 imagen"
                    
                    print(f"      - Comparación {fecha_real} ({objetivo_info}) vs {fecha_prev} ({previa_info}): {num_poligonos} polígonos")
                    
                except Exception as e:
                    print(f"    Error procesando comparación {fecha_real} vs {fecha_prev}: {str(e)}")
                    continue
            
            current_date += datetime.timedelta(days=1)
        
        return vectores_bloque
    
    def procesar_rango(self, start_year, start_month, end_year, end_month, export_folder='GEE_exports'):
        """
        Procesar un rango de fechas dividiéndolo en bloques
        
        Args:
            start_year (int): Año inicial
            start_month (int): Mes inicial (1-12)
            end_year (int): Año final
            end_month (int): Mes final (1-12)
            export_folder (str): Carpeta de exportación en Google Drive
        """
        print(f"🚀 Iniciando procesamiento desde {start_year}-{start_month:02d} hasta {end_year}-{end_month:02d}")
        print(f"📋 Procesando en bloques de {self.dias_por_bloque} días")
        print(f"🔄 Creando mosaicos para fechas objetivo e imágenes previas")
        print(f"🔄 Usando {self.num_imagenes_previas} fechas previas para comparación")
        
        # Fechas de inicio y fin
        fecha_inicio_total = datetime.date(start_year, start_month, 1)
        if end_month == 12:
            fecha_fin_total = datetime.date(end_year + 1, 1, 1) - datetime.timedelta(days=1)
        else:
            fecha_fin_total = datetime.date(end_year, end_month + 1, 1) - datetime.timedelta(days=1)
        
        # Procesar por bloques
        current_start = fecha_inicio_total
        bloque_num = 1
        
        while current_start <= fecha_fin_total:
            # Calcular fin del bloque actual
            current_end = min(
                current_start + datetime.timedelta(days=self.dias_por_bloque - 1),
                fecha_fin_total
            )
            
            print(f"\n🔄 BLOQUE {bloque_num}: {current_start} a {current_end}")
            
            # Procesar bloque
            vectores_bloque = self.procesar_bloque(current_start, current_end)
            
            # Exportar resultados del bloque si hay datos
            if vectores_bloque:
                vectores_combinados = ee.FeatureCollection(vectores_bloque).flatten()
                total_poligonos = vectores_combinados.size().getInfo()
                
                if total_poligonos > 0:
                    print(f"  ✅ Bloque completado: {total_poligonos} polígonos detectados")
                    
                    # Crear nombre de archivo descriptivo
                    start_str = current_start.strftime('%Y%m%d')
                    end_str = current_end.strftime('%Y%m%d')
                    description = f'Bloque_{start_str}_a_{end_str}'
                    
                    try:
                        task = ee.batch.Export.table.toDrive(
                            collection=vectores_combinados,
                            description=description,
                            folder=export_folder,
                            fileFormat='SHP'
                        )
                        
                        task.start()
                        print(f"  📤 Exportación iniciada: {description}")
                        
                    except Exception as e:
                        print(f"  ❌ Error en exportación del bloque: {str(e)}")
                else:
                    print(f"  📄 Bloque sin polígonos para exportar")
            else:
                print(f"  📄 No se encontraron áreas quemadas en este bloque")
            
            # Avanzar al siguiente bloque
            current_start = current_end + datetime.timedelta(days=1)
            bloque_num += 1
            
            # Esperar un poco entre bloques
            time.sleep(2)
        
        print(f"\n🎉 ¡Procesamiento completado! Se procesaron {bloque_num-1} bloques.")
        print("📋 Revisa tu Google Drive para los archivos exportados.")


# Ejemplo de uso
if __name__ == "__main__":
    # Inicializar detector
    detector = QuemadaDetector(
        region_asset_path='projects/ee-ezabaleta/assets/Zona',
        max_dias_busqueda=3,
        num_imagenes_previas=4,
        nubes_max_objetivo=100,    # Máximo 100% de nubes para imágenes objetivo
        nubes_max_previas=50,     # Máximo 50% de nubes para imágenes previas
        dias_por_bloque=4     # Procesar en bloques de 5 días
    )
    
    # Procesar rango de fechas
    # Ejemplo: procesar desde enero 2025 hasta junio 2025
    detector.procesar_rango(
        start_year=2025,
        start_month=9,
        end_year=2025,
        end_month=9,
        export_folder='GEE_exports'
    )


Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Detector inicializado. Región cargada con 1 features
Filtro nubes objetivo: < 100%
Filtro nubes previas: < 50%
Días por bloque: 4
🚀 Iniciando procesamiento desde 2025-09 hasta 2025-09
📋 Procesando en bloques de 4 días
🔄 Creando mosaicos para fechas objetivo e imágenes previas
🔄 Usando 4 fechas previas para comparación

🔄 BLOQUE 1: 2025-09-01 a 2025-09-04
  📅 Procesando bloque: 2025-09-01 a 2025-09-04
    Procesando mosaico objetivo del 2025-09-01 (2 imágenes)
    Creando mosaicos para 4 fechas previas...
      Fechas únicas encontradas: 4 de 4 solicitadas
        - 2025-08-22: 1 imagen
        - 2025-08-20: 2 imágenes → mosaico
        - 2025-08-12: 2 imágenes → mosaico
        - 2025-08-08: 1 imagen
    Creados 4 mosaicos previos
      - Comparación 2025-09-01 (mosaico 2 imgs) vs 2025-08-22 (1 imagen): 0 polígonos
      - Comparación 2025-09-01 (mosaico 2 imgs) vs 2025-08-20 (mosaico 2 imgs): 1 polígonos
      - Comparación 2025-09-01 (mosaico 2 imgs) vs 2025-08-12 (mosaico 2 imgs): 0

# Filtro areas quemadas por area y conversion a shp

En este codigo se unen todos los bloques calculados en el script anterior, se filtran por area y se guarda en local

In [6]:
import geopandas as gpd
import pandas as pd 
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

def merge_shp_files(input_folder):
    """
    Función para hacer merge de todos los archivos SHP en una carpeta
    manteniendo solo los campos fecha_obje y fecha_prev
    """
    print("🔄 Iniciando merge de archivos SHP...")
    
    # Buscar todos los archivos .shp en la carpeta
    shp_files = list(Path(input_folder).glob("*.shp"))
    
    if not shp_files:
        print("❌ No se encontraron archivos SHP en la carpeta especificada")
        return None
    
    print(f"📁 Encontrados {len(shp_files)} archivos SHP")
    
    # Lista para almacenar los GeoDataFrames
    gdfs = []
    
    for shp_file in shp_files:
        try:
            print(f"  📄 Procesando: {shp_file.name}")
            gdf = gpd.read_file(shp_file)
            
            # Verificar si las columnas existen
            if 'fecha_obje' in gdf.columns and 'fecha_prev' in gdf.columns:
                # Mantener solo las columnas necesarias y la geometría
                gdf_filtered = gdf[['fecha_obje', 'fecha_prev', 'geometry']].copy()
                gdfs.append(gdf_filtered)
                print(f"    ✅ Agregado: {len(gdf_filtered)} features")
            else:
                print(f"    ⚠️  Columnas faltantes en {shp_file.name}")
                print(f"    Columnas disponibles: {list(gdf.columns)}")
                
        except Exception as e:
            print(f"    ❌ Error procesando {shp_file.name}: {str(e)}")
            continue
    
    if not gdfs:
        print("❌ No se pudieron procesar archivos SHP válidos")
        return None
    
    # Combinar todos los GeoDataFrames
    print("🔄 Combinando GeoDataFrames...")
    merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    # Asegurar que el CRS sea consistente
    if merged_gdf.crs is None:
        print("⚠️  CRS no definido, asignando EPSG:4326")
        merged_gdf = merged_gdf.set_crs('EPSG:4326')
    
    print(f"✅ Merge completado: {len(merged_gdf)} features totales")
    return merged_gdf

def calculate_area_hectares(gdf):
    """
    Calcular área en hectáreas
    """
    print("🔄 Calculando área en hectáreas...")
    
    # Si está en coordenadas geográficas, proyectar a un sistema métrico
    if gdf.crs.is_geographic:
        # Proyectar a Web Mercator para cálculo de área
        gdf_proj = gdf.to_crs('EPSG:3857')
        area_m2 = gdf_proj.geometry.area
    else:
        area_m2 = gdf.geometry.area
    
    # Convertir a hectáreas (1 ha = 10,000 m²)
    area_ha = area_m2 / 10000
    
    return area_ha

def process_burned_areas(input_folder, output_file):
    """
    Función principal para procesar las áreas quemadas siguiendo el workflow especificado
    """
    print("🚀 Iniciando procesamiento de áreas quemadas")
    print(f"📂 Carpeta de entrada: {input_folder}")
    print(f"💾 Archivo de salida: {output_file}")
    print("-" * 60)
    
    # Paso 1: Merge de archivos SHP
    merged_gdf = merge_shp_files(input_folder)
    if merged_gdf is None:
        return
    
    print(f"\n📊 Estadísticas iniciales:")
    print(f"  - Features totales: {len(merged_gdf)}")
    print(f"  - CRS: {merged_gdf.crs}")
    
    # Paso 2: Crear campo de área y eliminar polígonos < 0.1 ha
    print("\n🔄 Paso 1: Calculando área y filtrando polígonos < 0.1 ha...")
    merged_gdf['area_ha'] = calculate_area_hectares(merged_gdf)
    
    # Mostrar estadísticas antes del filtro
    print(f"  📊 Área total inicial: {merged_gdf['area_ha'].sum():.2f} ha")
    print(f"  📊 Polígonos < 0.1 ha: {len(merged_gdf[merged_gdf['area_ha'] < 0.1])}")
    
    # Filtrar polígonos >= 0.1 ha
    filtered_gdf = merged_gdf[merged_gdf['area_ha'] >= 0.1].copy()
    print(f"  ✅ Polígonos después del filtro: {len(filtered_gdf)}")
    print(f"  📊 Área total después del filtro: {filtered_gdf['area_ha'].sum():.2f} ha")
    
    # Paso 3: Dissolve por fecha_obje sin crear multiparts
    print("\n🔄 Paso 2: Dissolve por fecha_obje...")
    print(f"  📊 Fechas únicas: {filtered_gdf['fecha_obje'].nunique()}")
    
    # Realizar dissolve por fecha_obje
    dissolved_gdf = filtered_gdf.dissolve(by='fecha_obje', as_index=False)
    
    # Explotar multiparts si existen
    dissolved_gdf = dissolved_gdf.explode(index_parts=False).reset_index(drop=True)
    
    print(f"  ✅ Polígonos después del dissolve: {len(dissolved_gdf)}")
    
    # Paso 4: Recalcular área y eliminar polígonos < 0.1 ha otra vez
    print("\n🔄 Paso 3: Recalculando área y filtrando < 0.1 ha...")
    dissolved_gdf['area_ha'] = calculate_area_hectares(dissolved_gdf)
    
    # Mostrar estadísticas antes del segundo filtro
    print(f"  📊 Área total antes del segundo filtro: {dissolved_gdf['area_ha'].sum():.2f} ha")
    print(f"  📊 Polígonos < 0.1 ha: {len(dissolved_gdf[dissolved_gdf['area_ha'] < 0.1])}")
    
    # Segundo filtro
    final_filtered_gdf = dissolved_gdf[dissolved_gdf['area_ha'] >= 0.1].copy()
    print(f"  ✅ Polígonos después del segundo filtro: {len(final_filtered_gdf)}")
    print(f"  📊 Área total después del segundo filtro: {final_filtered_gdf['area_ha'].sum():.2f} ha")
    
    # Paso 5: Aplicar buffer de 0.1 metros
    print("\n🔄 Paso 4: Aplicando buffer de 0.1 metros...")
    
    # Si está en coordenadas geográficas, proyectar temporalmente
    if final_filtered_gdf.crs.is_geographic:
        # Proyectar a Web Mercator para el buffer
        buffered_gdf = final_filtered_gdf.to_crs('EPSG:3857')
        buffered_gdf['geometry'] = buffered_gdf.geometry.buffer(0.1)
        # Volver al CRS original
        buffered_gdf = buffered_gdf.to_crs(final_filtered_gdf.crs)
    else:
        buffered_gdf = final_filtered_gdf.copy()
        buffered_gdf['geometry'] = buffered_gdf.geometry.buffer(0.1)
    
    # Paso 6: Recalcular área final
    print("\n🔄 Paso 5: Recalculando área final...")
    buffered_gdf['area_ha'] = calculate_area_hectares(buffered_gdf)
    
    print(f"  ✅ Área total final: {buffered_gdf['area_ha'].sum():.2f} ha")
    print(f"  📊 Polígonos finales: {len(buffered_gdf)}")
    
    # Paso 7: Guardar resultado
    print(f"\n💾 Guardando resultado en: {output_file}")
    
    # Crear directorio si no existe
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Guardar archivo
    buffered_gdf.to_file(output_file, driver='ESRI Shapefile')
    
    print("✅ Procesamiento completado exitosamente!")
    
    # Mostrar resumen final
    print("\n📊 RESUMEN FINAL:")
    print(f"  - Polígonos procesados: {len(buffered_gdf)}")
    print(f"  - Área total: {buffered_gdf['area_ha'].sum():.2f} ha")
    print(f"  - Fechas únicas: {buffered_gdf['fecha_obje'].nunique()}")
    print(f"  - Rango de fechas: {buffered_gdf['fecha_obje'].min()} a {buffered_gdf['fecha_obje'].max()}")
    print(f"  - Área promedio por polígono: {buffered_gdf['area_ha'].mean():.2f} ha")
    print(f"  - Área máxima: {buffered_gdf['area_ha'].max():.2f} ha")
    print(f"  - Área mínima: {buffered_gdf['area_ha'].min():.2f} ha")
    
    return buffered_gdf

# Configuración y ejecución
if __name__ == "__main__":
    # Configurar rutas
    input_folder = r"G:\Mi unidad\GEE_exports"
    output_file = r"G:\Mi unidad\GEE_exports\areas_quemadas_procesadas.shp"
    
    # Verificar que la carpeta existe
    if not os.path.exists(input_folder):
        print(f"❌ La carpeta {input_folder} no existe")
        print("Por favor, verifica la ruta y vuelve a intentar")
    else:
        # Ejecutar procesamiento
        resultado = process_burned_areas(input_folder, output_file)
        
        if resultado is not None:
            print(f"\n🎉 ¡Procesamiento completado exitosamente!")
            print(f"📁 Archivo guardado en: {output_file}")
        else:
            print("❌ El procesamiento falló")

🚀 Iniciando procesamiento de áreas quemadas
📂 Carpeta de entrada: G:\Mi unidad\GEE_exports
💾 Archivo de salida: G:\Mi unidad\GEE_exports\areas_quemadas_procesadas.shp
------------------------------------------------------------
🔄 Iniciando merge de archivos SHP...
📁 Encontrados 72 archivos SHP
  📄 Procesando: Bloque_20200101_a_20200104.shp
    ✅ Agregado: 216 features
  📄 Procesando: Bloque_20200105_a_20200108.shp
    ✅ Agregado: 165 features
  📄 Procesando: Bloque_20200109_a_20200112.shp
    ✅ Agregado: 244 features
  📄 Procesando: Bloque_20200113_a_20200116.shp
    ✅ Agregado: 718 features
  📄 Procesando: Bloque_20200117_a_20200120.shp
    ✅ Agregado: 457 features
  📄 Procesando: Bloque_20200125_a_20200128.shp
    ✅ Agregado: 180 features
  📄 Procesando: Bloque_20200129_a_20200201.shp
    ✅ Agregado: 138 features
  📄 Procesando: Bloque_20200202_a_20200205.shp
    ✅ Agregado: 91 features
  📄 Procesando: Bloque_20200206_a_20200209.shp
    ✅ Agregado: 223 features
  📄 Procesando: Bloque

# Deteccion areas quemadas diarias

En este codigo se utiliza la logica de bandas en imagenes satelitales para definir poligonos que representen areas quemadas para el dia actual, en caso que no se encuentren imagenes disponibles para el dia se omite el proceso. Tambien se actualiza la capa de incendios detectados en ArcGis Web. El script funciona con autenticacion de google earth engine asi que se tiene que crear un usuario y projecto con acceso a la API de google earth engine en google cloud

In [3]:
import ee
import datetime
from dateutil.relativedelta import relativedelta
import time
import os
import arcpy
import requests

# Inicializar Earth Engine
ee.Authenticate(force=True)
ee.Initialize(project='incendios-461918')

def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask_qa = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    scl = image.select('SCL')
    mask_scl = (scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))
    mask_final = mask_qa.And(mask_scl)
    return image.updateMask(mask_final)

def calcular_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

def calcular_bais2(image):
    b4 = image.select('B4')
    b6 = image.select('B6')
    b7 = image.select('B7')
    b8a = image.select('B8A')
    b12 = image.select('B12')
    bais2 = image.expression('(1 - sqrt((RE2 * RE3 * N2) / R)) * (sqrt((S2 - N2) / (S2 + N2)) + 1)', {
        'RE2': b6, 'RE3': b7, 'N2': b8a, 'R': b4, 'S2': b12
    }).rename('BAIS2')
    return bais2

def calcular_mirbi(image):
    return image.expression('10 * (B12 - (B11 + B8A) / 2)', {
        'B12': image.select('B12'), 'B11': image.select('B11'), 'B8A': image.select('B8A')
    }).rename('MIRBI')

def crear_mosaico_por_fecha(coleccion, fecha_inicio, fecha_fin, region):
    """
    Crea un mosaico de todas las imágenes disponibles en una fecha específica
    
    Args:
        coleccion: ImageCollection de Sentinel-2
        fecha_inicio: ee.Date de inicio del día
        fecha_fin: ee.Date de fin del día
        region: Geometría de la región de interés
    
    Returns:
        ee.Image: Mosaico de las imágenes del día o None si no hay imágenes
    """
    # Filtrar imágenes por fecha
    imagenes_fecha = (coleccion
                     .filterDate(fecha_inicio, fecha_fin)
                     .filterBounds(region))
    
    # Verificar si hay imágenes
    num_imagenes = imagenes_fecha.size()
    
    try:
        num_imagenes_info = num_imagenes.getInfo()
        if num_imagenes_info == 0:
            return None, 0
    except:
        return None, 0
    
    # Crear mosaico (las imágenes más recientes tendrán prioridad)
    mosaico = (imagenes_fecha
               .sort('system:time_start', False)  # Ordenar por fecha descendente
               .mosaic()
               .clip(region))
    
    # Agregar metadatos del mosaico
    fecha_str = fecha_inicio.format('YYYY-MM-dd').getInfo()
    mosaico = mosaico.set({
        'system:time_start': fecha_inicio.millis(),
        'fecha_mosaico': fecha_str,
        'num_imagenes_mosaico': num_imagenes_info
    })
    
    return mosaico, num_imagenes_info

def obtener_fechas_unicas_previas(coleccion, fecha_limite, num_fechas_max=10):
    """
    Obtiene las fechas únicas disponibles en la colección antes de una fecha límite
    
    Args:
        coleccion: ImageCollection
        fecha_limite: ee.Date límite (no incluida)
        num_fechas_max: Número máximo de fechas a retornar
    
    Returns:
        List: Lista de fechas únicas en formato ee.Date
    """
    # Filtrar imágenes antes de la fecha límite
    imagenes_previas = (coleccion
                       .filter(ee.Filter.lt('system:time_start', fecha_limite.millis()))
                       .sort('system:time_start', False))
    
    # Función para extraer fecha (YYYY-MM-DD) de una imagen
    def extraer_fecha(image):
        fecha = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return image.set('fecha_str', fecha)
    
    # Agregar campo de fecha string
    imagenes_con_fecha = imagenes_previas.map(extraer_fecha)
    
    # Obtener fechas únicas usando distinct
    fechas_unicas = imagenes_con_fecha.distinct('fecha_str').limit(num_fechas_max)
    
    # Convertir a lista para poder iterar
    lista_fechas = fechas_unicas.toList(num_fechas_max)
    
    # Extraer las fechas como ee.Date
    fechas_ee = []
    try:
        num_fechas = fechas_unicas.size().getInfo()
        print(f"📅 Encontradas {num_fechas} fechas únicas para procesar")
        
        for i in range(num_fechas):
            imagen = ee.Image(lista_fechas.get(i))
            fecha_ee = ee.Date(imagen.get('system:time_start'))
            # Truncar a inicio del día
            fecha_inicio_dia = fecha_ee.update(None, None, None, 0, 0, 0)
            fechas_ee.append(fecha_inicio_dia)
            
            # Mostrar fecha para debug
            fecha_str = fecha_ee.format('YYYY-MM-dd').getInfo()
            print(f"   📆 Fecha {i+1}: {fecha_str}")
            
    except Exception as e:
        print(f"❌ Error al obtener fechas únicas: {e}")
        return []
    
    return fechas_ee

# [Aquí van todas las funciones auxiliares sin cambios]
def obtener_url_feature_service(item_id, usuario, clave):
    """
    Obtiene la URL correcta del feature service desde ArcGIS Online
    """
    portal_url = "https://www.arcgis.com"
    
    # URL para obtener información del item
    item_info_url = f"{portal_url}/sharing/rest/content/items/{item_id}"
    
    # Parámetros para la solicitud
    params = {
        'f': 'json',
        'token': None  # Se obtendrá el token después
    }
    
    # Obtener token de autenticación
    token_url = f"{portal_url}/sharing/rest/generateToken"
    token_params = {
        'username': usuario,
        'password': clave,
        'referer': portal_url,
        'f': 'json'
    }
    
    try:
        # Solicitar token
        token_response = requests.post(token_url, data=token_params)
        token_data = token_response.json()
        
        if 'token' not in token_data:
            raise Exception(f"Error al obtener token: {token_data}")
        
        token = token_data['token']
        params['token'] = token
        
        # Obtener información del item
        item_response = requests.get(item_info_url, params=params)
        item_data = item_response.json()
        
        if 'url' not in item_data:
            raise Exception(f"No se encontró URL en el item: {item_data}")
        
        # Construir URL del feature service
        base_url = item_data['url']
        if not base_url.endswith('/'):
            base_url += '/'
        
        feature_service_url = f"{base_url}0"  # Capa 0
        
        return feature_service_url, token
        
    except Exception as e:
        raise Exception(f"Error al obtener URL del feature service: {e}")

def limpiar_archivos_temporales(output_folder_local, nombre_shp):
    """
    Elimina los archivos temporales generados durante el procesamiento
    """
    print("🧹 Limpiando archivos temporales...")
    
    # Lista de archivos a eliminar
    archivos_a_eliminar = [
        nombre_shp,  # Archivo original
        nombre_shp.replace('.shp', '_filtrado.shp'),
        nombre_shp.replace('.shp', '_procesado.shp'),
        nombre_shp.replace('.shp', '_filtrado_2.shp'),
        nombre_shp.replace('.shp', '_buffered.shp')
    ]
    
    for archivo_shp in archivos_a_eliminar:
        try:
            archivo_path = os.path.join(output_folder_local, archivo_shp)
            
            # Eliminar todos los archivos relacionados con el shapefile
            extensiones = ['.shp', '.shx', '.dbf', '.prj', '.cpg', '.xml']
            
            for ext in extensiones:
                archivo_completo = archivo_path.replace('.shp', ext)
                if os.path.exists(archivo_completo):
                    os.remove(archivo_completo)
                    print(f"   🗑️ Eliminado: {os.path.basename(archivo_completo)}")
            
        except Exception as e:
            print(f"   ⚠️ No se pudo eliminar {archivo_shp}: {e}")
    
    print("✅ Limpieza de archivos temporales completada")

def agregar_a_arcgis_web(shp_buffered, output_folder_local, nombre_shp):
    """
    Función para agregar shapefile a ArcGIS Online con pausas para estabilidad
    """
    portal_url = "https://www.arcgis.com/"
    usuario = "unidad.gestion.riesgo"
    clave = "XXXXXXX"
    item_id = "7b561988344046d4827141dcf8014b25"
    
    print("🔐 Iniciando sesión en ArcGIS Online...")
    arcpy.SignInToPortal(portal_url, usuario, clave)
    
    # Pausa después del login
    time.sleep(3)
    print("✅ Sesión iniciada correctamente")
    
    # Obtener URL correcta del feature service
    try:
        feature_service_url, token = obtener_url_feature_service(item_id, usuario, clave)
        print(f"🔗 Feature layer URL detectado: {feature_service_url}")
    except Exception as e:
        print(f"❌ Error al obtener URL del servicio: {e}")
        raise
    
    # Pausa antes de verificar el shapefile
    time.sleep(2)
    
    # Verificar que el shapefile esté completamente disponible
    print("📋 Verificando disponibilidad del shapefile...")
    if not arcpy.Exists(shp_buffered):
        raise Exception(f"El shapefile {shp_buffered} no existe o no está disponible")
    
    # Pausa adicional para asegurar que el archivo esté completamente creado
    time.sleep(5)
    
    try:
        print("📤 Iniciando carga de datos al servicio web...")
        arcpy.management.Append(inputs=shp_buffered,
                                 target=feature_service_url,
                                 schema_type="NO_TEST",
                                 field_mapping="",
                                 subtype="",
                                 expression="")
        
        # Pausa después de la carga
        time.sleep(3)
        print("✅ Datos agregados exitosamente al servicio web")
        
        # Limpiar archivos temporales después de subida exitosa
        limpiar_archivos_temporales(output_folder_local, nombre_shp)
        
        return True
        
    except Exception as e:
        print(f"❌ Error al agregar los datos al feature layer: {e}")
        raise

def esperar_archivo_disponible(archivo_path, max_intentos=30, pausa_entre_intentos=2):
    """
    Espera hasta que el archivo esté completamente disponible para ArcGIS
    """
    for intento in range(max_intentos):
        try:
            # Verificar existencia básica
            if not os.path.exists(archivo_path):
                print(f"⌛ Intento {intento + 1}: Archivo no existe aún...")
                time.sleep(pausa_entre_intentos)
                continue
            
            # Verificar que ArcGIS pueda acceder al archivo
            if arcpy.Exists(archivo_path):
                # Intentar describir el archivo para verificar acceso completo
                desc = arcpy.Describe(archivo_path)
                if desc.dataType == 'ShapeFile':
                    print(f"✅ Archivo disponible después de {intento + 1} intentos")
                    return True
            
            print(f"⌛ Intento {intento + 1}: Archivo no completamente disponible...")
            time.sleep(pausa_entre_intentos)
            
        except Exception as e:
            print(f"⌛ Intento {intento + 1}: Error de acceso - {e}")
            time.sleep(pausa_entre_intentos)
    
    return False

def post_procesar_shapefile(output_folder_local, nombre_shp):
    """
    Función para postprocesar shapefile con pausas entre operaciones
    """
    print("🔧 Iniciando postprocesamiento del shapefile...")
    
    # Configurar entorno ArcGIS
    arcpy.env.workspace = output_folder_local
    arcpy.env.overwriteOutput = True
    
    # Definir rutas de archivos
    shp_original = os.path.join(output_folder_local, nombre_shp)
    shp_filtrado = os.path.join(output_folder_local, nombre_shp.replace('.shp', '_filtrado.shp'))
    shp_dissolved = os.path.join(output_folder_local, nombre_shp.replace('.shp', '_procesado.shp'))
    shp_filtrado_2 = os.path.join(output_folder_local, nombre_shp.replace('.shp', '_filtrado_2.shp'))
    shp_buffered = os.path.join(output_folder_local, nombre_shp.replace('.shp', '_buffered.shp'))

    # Verificar que el archivo original esté completamente disponible
    print("🔍 Verificando disponibilidad del archivo original...")
    if not esperar_archivo_disponible(shp_original):
        raise Exception(f"El archivo {shp_original} no está disponible después de múltiples intentos")
    
    # Pausa adicional para estabilidad
    time.sleep(3)

    # Paso 1: Agregar y calcular campo Area_ha en shapefile original
    print("📏 Paso 1: Calculando área inicial...")
    try:
        # Verificar campos existentes
        campos_existentes = [f.name for f in arcpy.ListFields(shp_original)]
        if 'Area_ha' not in campos_existentes:
            print("   ➕ Agregando campo Area_ha...")
            arcpy.AddField_management(shp_original, 'Area_ha', 'DOUBLE')
            time.sleep(3)  # Pausa después de agregar campo
            print("   ✅ Campo Area_ha agregado")
        else:
            print("   ℹ️ Campo Area_ha ya existe")
        
        print("   📐 Calculando geometría...")
        arcpy.CalculateGeometryAttributes_management(shp_original, [["Area_ha", "AREA_GEODESIC"]], area_unit="HECTARES")
        time.sleep(4)  # Pausa después del cálculo de geometría
        print("✅ Área inicial calculada")
        
    except Exception as e:
        print(f"❌ Error en Paso 1: {e}")
        raise

    # Paso 2: Filtrar por área >= 0.1 hectáreas
    print("🔍 Paso 2: Filtrando por área >= 0.1 hectáreas...")
    try:
        arcpy.MakeFeatureLayer_management(shp_original, 'temp_layer', '"Area_ha" >= 0.1')
        time.sleep(3)  # Pausa después de crear layer temporal
        
        arcpy.CopyFeatures_management('temp_layer', shp_filtrado)
        time.sleep(4)  # Pausa después de copiar features
        
        arcpy.Delete_management('temp_layer')
        time.sleep(2)  # Pausa después de eliminar layer temporal
        print("✅ Filtrado inicial completado")
        
    except Exception as e:
        print(f"❌ Error en Paso 2: {e}")
        raise

    # Paso 3: Dissolver polígonos
    print("🔄 Paso 3: Disolviendo polígonos...")
    try:
        arcpy.Dissolve_management(shp_filtrado, shp_dissolved, dissolve_field='fecha_obje', multi_part='SINGLE_PART')
        time.sleep(8)  # Pausa más larga después del dissolve (operación compleja)
        print("✅ Dissolve completado")
        
    except Exception as e:
        print(f"❌ Error en Paso 3: {e}")
        raise

    # Paso 4: Recalcular área después del dissolve
    print("📏 Paso 4: Recalculando área después del dissolve...")
    try:
        # Verificar campos existentes
        campos_existentes = [f.name for f in arcpy.ListFields(shp_dissolved)]
        if 'Area_ha' not in campos_existentes:
            print("   ➕ Agregando campo Area_ha...")
            arcpy.AddField_management(shp_dissolved, 'Area_ha', 'DOUBLE')
            time.sleep(3)  # Pausa después de agregar campo
            print("   ✅ Campo Area_ha agregado")
        else:
            print("   ℹ️ Campo Area_ha ya existe")
        
        print("   📐 Calculando geometría...")
        arcpy.CalculateGeometryAttributes_management(shp_dissolved, [["Area_ha", "AREA_GEODESIC"]], area_unit="HECTARES")
        time.sleep(4)  # Pausa después del cálculo de geometría
        print("✅ Área recalculada")
        
    except Exception as e:
        print(f"❌ Error en Paso 4: {e}")
        raise

    # Paso 5: Segundo filtro por área >= 0.1 hectáreas
    print("🔍 Paso 5: Segundo filtro por área >= 0.1 hectáreas...")
    try:
        arcpy.MakeFeatureLayer_management(shp_dissolved, 'temp_layer_2', '"Area_ha" >= 0.1')
        time.sleep(3)  # Pausa después de crear layer temporal
        
        arcpy.CopyFeatures_management('temp_layer_2', shp_filtrado_2)
        time.sleep(4)  # Pausa después de copiar features
        
        arcpy.Delete_management('temp_layer_2')
        time.sleep(2)  # Pausa después de eliminar layer temporal
        print("✅ Segundo filtrado completado")
        
    except Exception as e:
        print(f"❌ Error en Paso 5: {e}")
        raise

    # Paso 6: Crear buffer
    print("📐 Paso 6: Creando buffer de 0.1 metros...")
    try:
        arcpy.Buffer_analysis(shp_filtrado_2, shp_buffered, "0.1 Meters", dissolve_option="NONE", method="PLANAR")
        time.sleep(6)  # Pausa después del buffer (operación compleja)
        print("✅ Buffer creado")
        
    except Exception as e:
        print(f"❌ Error en Paso 6: {e}")
        raise

    # Paso 7: Calcular área final
    print("📏 Paso 7: Calculando área final...")
    try:
        # Verificar campos existentes
        campos_existentes = [f.name for f in arcpy.ListFields(shp_buffered)]
        if 'Area_ha' not in campos_existentes:
            print("   ➕ Agregando campo Area_ha...")
            arcpy.AddField_management(shp_buffered, 'Area_ha', 'DOUBLE')
            time.sleep(3)  # Pausa después de agregar campo
            print("   ✅ Campo Area_ha agregado")
        else:
            print("   ℹ️ Campo Area_ha ya existe")
        
        print("   📐 Calculando geometría...")
        arcpy.CalculateGeometryAttributes_management(shp_buffered, [["Area_ha", "AREA_GEODESIC"]], area_unit="HECTARES")
        time.sleep(4)  # Pausa después del cálculo de geometría
        print("✅ Área final calculada")
        
    except Exception as e:
        print(f"❌ Error en Paso 7: {e}")
        raise

    print(f"🎯 Procesamiento post-exportación completo: {shp_buffered}")
    
    # Pausa final antes de enviar a ArcGIS Online
    time.sleep(3)
    print("🌐 Enviando a ArcGIS Online...")
    
    try:
        agregar_a_arcgis_web(shp_buffered, output_folder_local, nombre_shp)
    except Exception as e:
        print(f"❌ Error al enviar a ArcGIS Online: {e}")
        raise

def procesar_imagen_diaria(region_asset_path, export_folder='GEE_exports_diaria'):
    """
    Función principal para procesar la imagen del día actual con mosaicos por fecha
    
    Args:
        region_asset_path (str): Ruta al asset de la región en GEE
        export_folder (str): Carpeta donde exportar los resultados
    """
    
    # Parámetros
    num_fechas_previas = 10  # Cambio: ahora es número de fechas únicas, no imágenes
    umbral_diff_ndvi = -0.3
    umbral_ndvi_previa = 0.5
    umbral_bais2 = 6
    umbral_mirbi = 0
    
    # Cargar región
    try:
        region = ee.FeatureCollection(region_asset_path)
        print(f"Región cargada: {region_asset_path}")
    except Exception as e:
        print(f"Error al cargar la región: {e}")
        return False
    
    # Obtener fecha de hoy
    fecha_hoy = datetime.datetime.now() - datetime.timedelta(days=1)
    fecha_hoy_str = fecha_hoy.strftime('%Y-%m-%d')
    print(f"Procesando para la fecha: {fecha_hoy_str}")
    
    # Crear fechas para Earth Engine
    fecha_ee_inicio = ee.Date(fecha_hoy_str)
    fecha_ee_fin = fecha_ee_inicio.advance(1, 'day')
    
    # Cargar colección Sentinel-2 SIN filtro de nubes para imagen objetivo
    coleccion = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(region)
                 .map(mask_s2_clouds)
                 .map(lambda img: img.clip(region)))
    
    # Cargar colección Sentinel-2 CON filtro de nubes para imágenes previas
    coleccion_prev = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                     .filterBounds(region)
                     .map(mask_s2_clouds)
                     .map(lambda img: img.clip(region)))
    
    print("Colecciones Sentinel-2 cargadas")
    
    # ===== CREAR MOSAICO PARA IMAGEN OBJETIVO =====
    print("🖼️ Creando mosaico para imagen objetivo...")
    mosaico_objetivo, num_img_objetivo = crear_mosaico_por_fecha(
        coleccion, fecha_ee_inicio, fecha_ee_fin, region
    )
    
    if mosaico_objetivo is None:
        print(f"❌ No hay imagen disponible para hoy ({fecha_hoy_str}). No se procesará.")
        return False
    
    print(f"✅ Mosaico objetivo creado con {num_img_objetivo} imagen(es)")
    
    # Calcular índices para la imagen objetivo
    ndvi_objetivo = calcular_ndvi(mosaico_objetivo)
    bais_current = calcular_bais2(mosaico_objetivo)
    mirbi_current = calcular_mirbi(mosaico_objetivo)
    
    # ===== OBTENER FECHAS ÚNICAS PREVIAS =====
    print("📅 Obteniendo fechas únicas previas...")
    fechas_previas = obtener_fechas_unicas_previas(
        coleccion_prev, 
        fecha_ee_inicio, 
        num_fechas_previas
    )
    
    if len(fechas_previas) == 0:
        print("❌ No se encontraron fechas previas para comparar")
        return False
    
    print(f"✅ Se procesarán {len(fechas_previas)} fechas únicas")
    
    # ===== PROCESAR CADA FECHA PREVIA =====
    vectores_finales = []
    
    for j, fecha_previa in enumerate(fechas_previas):
        print(f"\n🔄 Procesando fecha previa {j+1}/{len(fechas_previas)}")
        
        try:
            # Crear fechas de inicio y fin del día
            fecha_prev_inicio = fecha_previa
            fecha_prev_fin = fecha_previa.advance(1, 'day')
            
            # Crear mosaico para esta fecha
            print("   🖼️ Creando mosaico para fecha previa...")
            mosaico_prev, num_img_prev = crear_mosaico_por_fecha(
                coleccion_prev, fecha_prev_inicio, fecha_prev_fin, region
            )
            
            if mosaico_prev is None:
                print("   ❌ No se pudo crear mosaico para esta fecha")
                continue
            
            fecha_prev_str = fecha_prev_inicio.format('YYYY-MM-dd').getInfo()
            print(f"   ✅ Mosaico creado con {num_img_prev} imagen(es) para {fecha_prev_str}")
            
            # Calcular NDVI para mosaico previo
            ndvi_prev = calcular_ndvi(mosaico_prev)
            
            # Calcular diferencia NDVI
            diff_ndvi = ndvi_objetivo.subtract(ndvi_prev).rename('Diferencia_NDVI')
            
            # Crear máscara binaria
            zonas = diff_ndvi.lt(umbral_diff_ndvi).selfMask()
            zonas_clip = zonas.clip(region)
            
            # Reducir a vectores
            vectores = zonas_clip.reduceToVectors(
                geometry=region,
                scale=12.5,
                geometryType='polygon',
                eightConnected=True,
                labelProperty='zone',
                maxPixels=1e8
            )
            
            # Verificar si se generaron vectores
            num_vectores = vectores.size().getInfo()
            if num_vectores == 0:
                print(f"   ❌ No se generaron vectores iniciales para {fecha_prev_str}")
                continue
            
            print(f"   ✅ Se generaron {num_vectores} vectores iniciales para {fecha_prev_str}")
            
            # Unir polígonos
            union_poligono = vectores.geometry()
            poligono_current = ee.FeatureCollection([ee.Feature(union_poligono)])
            
            # DEBUG: Mostrar valores de los índices antes del filtrado
            print(f"   📊 Evaluando filtros combinados:")
            
            # Aplicar condición combinada CON DEBUG
            mask_bais2 = bais_current.lt(umbral_bais2)
            mask_mirbi = mirbi_current.gt(umbral_mirbi)
            mask_ndvi_prev = ndvi_prev.gt(umbral_ndvi_previa)
            
            # Verificar cada máscara por separado
            area_bais2 = mask_bais2.clip(poligono_current).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=poligono_current.geometry(),
                scale=12.5,
                maxPixels=1e8
            ).getInfo()
            
            area_mirbi = mask_mirbi.clip(poligono_current).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=poligono_current.geometry(),
                scale=12.5,
                maxPixels=1e8
            ).getInfo()
            
            area_ndvi = mask_ndvi_prev.clip(poligono_current).reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=poligono_current.geometry(),
                scale=12.5,
                maxPixels=1e8
            ).getInfo()
            
            print(f"      - BAIS2 < {umbral_bais2}: {area_bais2}")
            print(f"      - MIRBI > {umbral_mirbi}: {area_mirbi}")
            print(f"      - NDVI_prev > {umbral_ndvi_previa}: {area_ndvi}")
            
            # Combinar máscaras
            combined_mask = (mask_bais2.Or(mask_mirbi)
                            .And(mask_ndvi_prev)
                            .selfMask()
                            .rename('Combined_mask'))
            
            # Recortar máscara
            image_clip_union = combined_mask.clip(poligono_current)
            
            # Verificar si la máscara combinada tiene píxeles
            area_combinada = image_clip_union.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=poligono_current.geometry(),
                scale=12.5,
                maxPixels=1e8
            ).getInfo()
            
            print(f"      - Máscara combinada: {area_combinada}")
            
            if area_combinada.get('Combined_mask', 0) == 0:
                print(f"   ⚠️ La máscara combinada está vacía para {fecha_prev_str}")
                print(f"   💡 Sugerencia: Ajustar umbrales - BAIS2:{umbral_bais2}, MIRBI:{umbral_mirbi}, NDVI:{umbral_ndvi_previa}")
                continue
            
            # Reducir a vectores finales
            vectores_final = image_clip_union.reduceToVectors(
                geometry=region,
                scale=12.5,
                geometryType='polygon',
                eightConnected=True,
                maxPixels=1e8
            )
            
            # Verificar vectores finales
            num_vectores_final = vectores_final.size().getInfo()
            if num_vectores_final == 0:
                print(f"   ⚠️ No se generaron vectores finales para {fecha_prev_str}")
                continue
            
            print(f"   ✅ Se generaron {num_vectores_final} vectores finales para {fecha_prev_str}")
            
            # Agregar metadatos
            vectores_final = vectores_final.map(lambda feature: feature.set({
                'fecha_objetivo': fecha_hoy_str,
                'fecha_previa': fecha_prev_str,
                'indice_comparacion': j,
                'num_img_objetivo': num_img_objetivo,
                'num_img_previa': num_img_prev
            }))
            
            vectores_finales.append(vectores_final)
            
        except Exception as e:
            print(f"   ❌ Error procesando fecha previa {j+1}: {e}")
            continue
    
    # Combinar todos los vectores
    if vectores_finales:
        vectores_combinados = ee.FeatureCollection(vectores_finales).flatten()
        
        # Verificar si hay resultados
        num_resultados = vectores_combinados.size().getInfo()
        if num_resultados > 0:
            # Exportar
            fecha_limpia = fecha_hoy_str.replace('-', '_')
            descripcion = f'Vectores_Mosaicos_{fecha_limpia}'
            
            task = ee.batch.Export.table.toDrive(
                collection=vectores_combinados,
                description=descripcion,
                folder=export_folder,
                fileFormat='SHP'
            )
            
            task.start()
            print(f"\n🚀 Exportación iniciada: {descripcion}")
            print(f"📊 Número total de features a exportar: {num_resultados}")
            print(f"📅 Fechas procesadas: {len(fechas_previas)}")
            
            # Esperar que el usuario sincronice el archivo SHP en su carpeta local
            nombre_shp = f'Vectores_Mosaicos_{fecha_limpia}.shp'
            carpeta_local = r"G:\Mi unidad\GEE_exports_diaria"

            print("\n⏳ Esperando a que el archivo esté disponible localmente para su procesamiento...")

            # Esperar a que aparezca el archivo SHP en la carpeta
            shp_path = os.path.join(carpeta_local, nombre_shp)
            espera_max = 300  # 5 minutos
            tiempo_espera = 0

            while not os.path.exists(shp_path) and tiempo_espera < espera_max:
                time.sleep(10)
                tiempo_espera += 10
                print(f"   ⌛ Esperando archivo: {nombre_shp} ({tiempo_espera}s)")

            if os.path.exists(shp_path):
                try:
                    # Agregar metadatos al shapefile antes del procesamiento
                    print(f"\n📋 Archivo encontrado, iniciando postprocesamiento...")
                    post_procesar_shapefile(carpeta_local, nombre_shp)
                except Exception as e:
                    print(f"❌ Error durante el postprocesamiento: {e}")
            else:
                print(f"⚠️ No se encontró el archivo {nombre_shp} después de {espera_max} segundos.")
            return True
        else:
            print("❌ No se encontraron áreas de cambio para exportar")
            return False
    else:
        print("❌ No se generaron vectores para ninguna fecha previa")
        return False

def main():
    """
    Función principal para ejecutar el análisis diario con mosaicos
    """
    # Configuración
    REGION_ASSET_PATH = 'projects/ee-ezabaleta/assets/Zona'  # Cambia por tu ruta
    EXPORT_FOLDER = 'GEE_exports_diaria'  # Carpeta en Google Drive
    
    print("="*70)
    print("ANÁLISIS DIARIO DE CAMBIOS CON MOSAICOS - GOOGLE EARTH ENGINE")
    print("="*70)
    print("🔧 Funcionalidades nuevas:")
    print("   • Mosaicos automáticos por fecha (múltiples imágenes → 1 mosaico)")
    print("   • Fechas únicas (sin duplicados)")
    print("   • Mejor cobertura espacial")
    print("   • Metadatos de número de imágenes por mosaico")
    print("="*70)
    
    try:
        resultado = procesar_imagen_diaria(REGION_ASSET_PATH, EXPORT_FOLDER)
        
        if resultado:
            print("\n✅ Procesamiento completado exitosamente")
            print("🎯 Beneficios obtenidos:")
            print("   • Mayor cobertura temporal (más fechas únicas)")
            print("   • Mejor cobertura espacial (mosaicos completos)")
            print("   • Eliminación de duplicados por fecha")
            print("   • Metadatos detallados de composición de mosaicos")
        else:
            print("\n❌ Procesamiento terminado sin resultados")
            
    except Exception as e:
        print(f"\n❌ Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
    
    print("="*70)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'arcpy'

# Script calculo Severidad por poligonos

Este codigo realiza el calculo de severidad en uno o varios poligonos cargados utilizando la informacion de las bandas en imagenes sentinel, este script guarda la informacion en rasters independientes para cada id de poligono

In [ ]:
from numpy import gcd
import ee
import datetime
from dateutil.relativedelta import relativedelta
import time
import os
import arcpy
import geopandas as gpd

# Inicializar Earth Engine
ee.Authenticate(force=True)
ee.Initialize(project='incendios-461918')

# Cargar capa de polígonos (GeoJSON, Shapefile, etc.)
gdf = gpd.read_file("E:\ERLUAN\Amenas_Incendios\Vectores\Finales_puntos\Vectores_prueba_severidad.shp")  # Asegúrate que tenga columna 'fecha' y geometría

# Parámetros
dias_rango = 60
max_nubes = 50
max_nubes_poligono = 10  # Porcentaje máximo de nubes permitido dentro del polígono

# Función para enmascarar nubes
def maskS2sr(image):
    qa = image.select('QA60')
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, image.propertyNames())

# Función para verificar nubes dentro del polígono
def check_clouds_in_polygon(image, geometry):
    qa = image.select('QA60')
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)
    
    # Crear máscara de nubes y cirrus
    cloud_mask = qa.bitwiseAnd(cloudBitMask).neq(0).Or(qa.bitwiseAnd(cirrusBitMask).neq(0))
    
    # Calcular el porcentaje de nubes en el polígono
    area_total = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=10,
        maxPixels=1e13
    ).get('area')
    
    area_nubes = ee.Image.pixelArea().updateMask(cloud_mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=10,
        maxPixels=1e13
    ).get('area')
    
    # Manejar casos donde no hay nubes (area_nubes sería null)
    area_nubes = ee.Number(area_nubes).divide(ee.Number(area_total)).multiply(100)
    porcentaje_nubes = ee.Number(ee.Algorithms.If(area_nubes, area_nubes, 0))
    
    return porcentaje_nubes

# Función para obtener primer mosaico válido (sin nubes en el polígono)
def get_mosaico_valid(geometry, start_date, end_date):
    date_list = ee.List.sequence(0, ee.Date(end_date).difference(ee.Date(start_date), 'day').subtract(1))
    fechas = date_list.map(lambda d: ee.Date(start_date).advance(d, 'day'))

    def iterate_func(fecha, result):
        result = ee.Dictionary(result)
        found = result.get('encontrado')
        fecha = ee.Date(fecha)

        # Filtrar imágenes del día
        coleccion_dia = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(geometry) \
            .filterDate(fecha, fecha.advance(1, 'day')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_nubes))

        # Verificar si hay imágenes disponibles
        cond_imagenes = coleccion_dia.size().gt(0)

        def check_polygon_clouds():
            # Tomar la primera imagen para verificar nubes en el polígono
            primera_imagen = ee.Image(coleccion_dia.first())
            porcentaje_nubes_poligono = check_clouds_in_polygon(primera_imagen, geometry)
            
            # Verificar si el porcentaje de nubes en el polígono es aceptable
            imagen_valida = porcentaje_nubes_poligono.lt(max_nubes_poligono)
            
            return ee.Dictionary({
                'encontrado': True, 
                'fecha': fecha,
                'porcentaje_nubes': porcentaje_nubes_poligono,
                'imagen_valida': imagen_valida
            })

        return ee.Algorithms.If(found,
            result,
            ee.Algorithms.If(cond_imagenes,
                ee.Algorithms.If(
                    ee.Dictionary(check_polygon_clouds()).get('imagen_valida'),
                    check_polygon_clouds(),
                    result
                ),
                result
            )
        )

    resultado = ee.List(fechas).iterate(iterate_func, ee.Dictionary({'encontrado': False}))
    resultado = ee.Dictionary(resultado)

    # Verificar si se encontró una imagen válida
    encontrado = resultado.get('encontrado')
    
    def create_mosaic():
        fecha_valida = ee.Date(resultado.get('fecha'))
        porcentaje_nubes_final = resultado.get('porcentaje_nubes')
        
        coleccion = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(geometry) \
            .filterDate(fecha_valida, fecha_valida.advance(1, 'day')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_nubes)) \
            .map(maskS2sr)

        mosaico = coleccion.mosaic().clip(geometry).set({
            'fecha_real': fecha_valida.format('YYYY-MM-dd'),
            'porcentaje_nubes_poligono': porcentaje_nubes_final,
            'imagen_encontrada': True
        })
        
        return mosaico
    
    def empty_image():
        return ee.Image().set({
            'fecha_real': 'No encontrada',
            'porcentaje_nubes_poligono': -1,
            'imagen_encontrada': False
        })
    
    return ee.Algorithms.If(encontrado, create_mosaic(), empty_image())

# Iterar sobre cada polígono
for idx, row in gdf.iterrows():
    fecha_str = row['fecha_obje']
    from shapely.geometry import MultiPolygon, Polygon

    # Dentro del bucle for
    if isinstance(row.geometry, MultiPolygon):
        geom_shapely = list(row.geometry.geoms)[0]  # Usa solo el primer polígono
    else:
        geom_shapely = row.geometry

    geom_coords = list(geom_shapely.exterior.coords)
    geom = ee.Geometry.Polygon([geom_coords])
    target_date = ee.Date(fecha_str)

    fecha_inicio_antes = target_date.advance(-dias_rango, 'day')
    fecha_fin_antes = target_date
    fecha_inicio_despues = target_date.advance(1, 'day')
    fecha_fin_despues = target_date.advance(dias_rango + 1, 'day')

    try:
        print(f'🔍 Procesando polígono {idx}, fecha {fecha_str}')
        
        mosaico_antes = get_mosaico_valid(geom, fecha_inicio_antes, fecha_fin_antes)
        mosaico_despues = get_mosaico_valid(geom, fecha_inicio_despues, fecha_fin_despues)

        # Verificar que ambas imágenes fueron encontradas
        try:
            imagen_antes_ok = ee.Image(mosaico_antes).get('imagen_encontrada').getInfo()
            imagen_despues_ok = ee.Image(mosaico_despues).get('imagen_encontrada').getInfo()
        except:
            # Si hay error al obtener la propiedad, asumir que no se encontraron imágenes
            imagen_antes_ok = False
            imagen_despues_ok = False
        
        # Solo proceder si ambas imágenes son válidas
        if imagen_antes_ok and imagen_despues_ok:
            try:
                # Obtener información de las nubes
                fecha_antes = ee.Image(mosaico_antes).get('fecha_real').getInfo()
                fecha_despues = ee.Image(mosaico_despues).get('fecha_real').getInfo()
                nubes_antes = ee.Image(mosaico_antes).get('porcentaje_nubes_poligono').getInfo()
                nubes_despues = ee.Image(mosaico_despues).get('porcentaje_nubes_poligono').getInfo()
                
                # Mostrar información de las imágenes encontradas
                print(f'  📅 Imagen ANTES: {fecha_antes} (nubes: {nubes_antes:.1f}%)')
                print(f'  📅 Imagen DESPUÉS: {fecha_despues} (nubes: {nubes_despues:.1f}%)')

                # Calcular dNBR
                def calc_nbr(img):
                    return ee.Image(img).normalizedDifference(['B8', 'B12']).rename('NBR')

                nbr_antes = calc_nbr(mosaico_antes)
                nbr_despues = calc_nbr(mosaico_despues)
                dnbr = nbr_antes.subtract(nbr_despues).multiply(1000).rename('dNBR')

                # Exportar a Drive
                export_id = f'dNBR_{idx}_{fecha_str}'
                task = ee.batch.Export.image.toDrive(
                    image=dnbr,
                    description=export_id,
                    folder='GEE_SENTINEL',
                    fileNamePrefix=export_id,
                    region=geom,
                    scale=10,
                    maxPixels=1e13,
                    crs='EPSG:4326'
                )
                task.start()
                print(f'✅ Exportando dNBR para polígono {idx}, fecha {fecha_str}')
                print(f'   Task ID: {task.id}')
                
                # Opcional: esperar un poco entre exportaciones para no sobrecargar
                import time
                time.sleep(2)
                
            except Exception as e_inner:
                print(f'⚠️ Error al procesar imágenes válidas para polígono {idx}: {e_inner}')
        else:
            if not imagen_antes_ok:
                print(f'⚠️ No se encontró imagen ANTES válida (sin nubes) para polígono {idx}')
            if not imagen_despues_ok:
                print(f'⚠️ No se encontró imagen DESPUÉS válida (sin nubes) para polígono {idx}')

    except Exception as e:
        print(f'⚠️ Error en polígono {idx}: {e}')